In [13]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Missing").getOrCreate()

In [14]:
spark

In [15]:
training=spark.read.csv("test1.csv", header=True, inferSchema=True)

In [16]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [17]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [18]:
## [Age, Experience]----->> New Feature----->Independent Feature

In [20]:
from pyspark.ml.feature import VectorAssembler
feature_assembler=VectorAssembler(inputCols=['age', 'Experience'], outputCol="Independent Features")

In [21]:
output=feature_assembler.transform(training)

In [22]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [25]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [30]:
finalized_data=output.select(['Independent Features', "Salary"])

In [31]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [37]:
from pyspark.ml.regression import LinearRegression

## Train Test Split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor=LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor=regressor.fit(train_data)

In [38]:
## Coefficients
regressor.coefficients

DenseVector([28.4757, 1271.3568])

In [39]:
## Intercepts
regressor.intercept

14299.832495812996

In [44]:
## Predictions
pred_results=regressor.evaluate(test_data)

In [45]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|         [31.0,10.0]| 30000|27896.147403685147|
+--------------------+------+------------------+



In [47]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(2103.852596314853, 4426195.747020748)